<a href="https://colab.research.google.com/github/kubre/ML-Practice/blob/master/Text_Color_Accessibility_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as pl
import IPython

In [0]:
COL_NAMES = ['Red', 'Green', 'Blue', 'Text']
TEXT_COLORS = ['Black', 'White']
df = pd.read_csv('/content/data/color.csv', names= COL_NAMES, header= 0)
# r g b - red green blue
# t - Text color 0 Black, 1 White

In [88]:
df.head()

,Red,Green,Blue,Text
0,0,0,0,1
1,0,0,51,1
2,0,0,102,1
3,0,0,153,1
4,0,0,204,1


In [0]:
df_train = df.sample(frac= 0.80, random_state= 0)
df_test = df.drop(df_train.index)

In [0]:
train_y = df_train.pop('Text')
test_y = df_test.pop('Text')

In [0]:
def input_fn(features, labels, training= True, batch_size= 256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

In [0]:
feature_cols = []

for key in df_train.keys():
  feature_cols.append(tf.feature_column.numeric_column(key= key))

In [0]:
cl = tf.estimator.LinearClassifier(
    feature_columns= feature_cols,
    n_classes=2
)
IPython.display.clear_output()

In [94]:
cl.train(
    input_fn= lambda: input_fn(df_train, train_y, training= True),
    steps= 8000
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp7ndmwdux/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931475, step = 0
INFO:tensorflow:global_step/sec: 495.06
INFO:tensorflow:loss = 1.8298018, step = 100 (0.204 sec)
INFO:tensorflow:global_step/sec: 707.552
INFO:tensorflow:loss = 1.2845516, step = 200 (0.144 sec)
INFO:tensorflow:global_step/sec: 717.957
INFO:tensorflow:loss = 0.9365027, step = 300 (0.137 sec)
INFO:tensorflow:global_step/sec: 761.969
INFO:tensorflow:loss = 0.6726479, step = 400 (0.133 sec)
INFO:tensorflow:global_step/sec: 695.391
INFO:tensorflow:loss = 0.7126256, step = 500 (0.142 sec)
INFO:te

In [95]:
result = cl.evaluate(input_fn= lambda: input_fn(df_test, test_y, training= False))
IPython.display.clear_output()
print('\nTest set Accuracy: {accuracy:0.3f}'.format(**result))


Test set Accuracy: 0.837


In [0]:
def input_fn_p(features, batch_size= 256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [97]:
r_val = 0 #@param { type: 'slider', min: 0, max: 255 }
g_val = 0 #@param { type: 'slider', min: 0, max: 255 }
b_val = 0 #@param { type: 'slider', min: 0, max: 255 }

predict = {
    'Red': [float(r_val)],
    'Green': [float(g_val)],
    'Blue': [float(b_val)]
}

predictions = cl.predict(input_fn= lambda: input_fn_p(predict))

txt_color = ""

for prediction in predictions:
  class_ids = prediction['class_ids'][0]
  probability = prediction['probabilities'][class_ids]
  txt_color = TEXT_COLORS[class_ids]
  IPython.display.clear_output()
  print(f'Text colour should be {TEXT_COLORS[class_ids]}: {probability * 100:.1f}%\nSample: ')

js_code = '''
var div = document.createElement("div");
div.style.width = "100px";
div.style.height = "30px";
div.style.background = "rgb({}, {}, {})";
div.style.paddingTop = "15px";
div.style.textAlign = "center";
div.style.color = "{}";
div.innerHTML = "TEXT";
document.querySelector("#output-area").appendChild(div);
'''.format(r_val, g_val, b_val, txt_color)
display(IPython.display.Javascript(js_code))

Text colour should be White: 99.9%
Sample: 


<IPython.core.display.Javascript object>